# PS4 EDA

## T MAC

### Dataset available at: *https://www.kaggle.com/shivamb/all-playstation-4-games*

# Import Data

In [1]:
import pandas as pd
import plotly.express as px
from scipy import stats
import wquantiles as wq
import matplotlib as plt

### Correct data types

In [2]:
data_import = pd.read_csv("playstation_4_games.csv")

data_import["ReleaseDate"] = pd.to_datetime(data_import["ReleaseDate"], format="%Y-%m-%d")
data_import["CompletionTime(Hours)"] = data_import["CompletionTime(Hours)"].str.replace('-', '').str.strip('+').astype('int64', errors='ignore')


data_import



,ID,GamePSID,GameName,Publisher,ReleaseYear,ReleaseDate,Developer,Genre,Features,Size,Medium,Hardware,CompletionTime(Hours),is_Digital_game,is_Physical_game,OfficialWebsite
0,1,3045,Q*Bert Rebooted,LOOT Interactive,2015.0,2015-02-17,Gonzo Games,Action,"PS Plus, PS Now",109.00,NaN,NaN,NaN,0,0,http://www.sidelineamusements.com/qbert/
1,2,3051,Q*Bert Rebooted (EU),LOOT Interactive,2015.0,2015-07-22,LOOT Interactive,Action,PS Now,127.00,Digital only,NaN,NaN,0,0,http://www.sidelineamusements.com/qbert/
2,3,6703,Q.U.B.E. 2,Trapped Nerve Games,2018.0,2018-03-13,Toxic Games,"Puzzle, Platformer",NaN,2682.88,Digital only,NaN,NaN,0,0,http://qube-game.com/
3,4,3403,Q.U.B.E. Director's Cut,GRIP Digital,2015.0,2015-07-21,Toxic Games,"Puzzle, Platformer",PS Plus,1945.60,Digital only,NaN,34,0,0,http://qube-game.com/
4,5,13176,Quake (PS4),Bethesda Softworks,2021.0,2021-08-19,"MachineGames, Nightdive Studios, id Software",First Person Shooter,NaN,NaN,Physical and Digital,NaN,NaN,0,0,https://bethesda.net/en/game/quake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9681,9682,10928,9th Dawn III,Valorware,2020.0,2020-10-06,Valorware,"Open World, Action-RPG, Role Playing",NaN,653.00,Physical and Digital,NaN,NaN,0,0,https://valorware.squarespace.com/9th-dawn-iii
9682,9683,10929,9th Dawn III (EU),Valorware,2020.0,2020-10-06,Valorware,"Open World, Action-RPG, Role Playing",NaN,653.00,Digital only,NaN,NaN,0,0,https://valorware.squarespace.com/9th-dawn-iii
9683,9684,7114,n Verlore Verstand,Skobbejak Games,2018.0,2018-06-12,Skobbejak Games,"Puzzle, Adventure, Platformer",NaN,2170.88,Digital only,NaN,NaN,0,0,http://skobbejakgames.com/n-verlore-verstand/
9684,9685,7110,n Verlore Verstand (EU),Skobbejak Games,2018.0,2018-06-12,Skobbejak Games,"Puzzle, Adventure, Platformer",NaN,2170.88,Digital only,NaN,NaN,0,0,http://skobbejakgames.com/n-verlore-verstand/


## EDA genre

In [3]:
#Splitiing genres
data_import["Genre"].str.split(',', expand=True)


,0,1,2,3,4,5,6,7
0,Action,None,None,None,None,None,None,None
1,Action,None,None,None,None,None,None,None
2,Puzzle,Platformer,None,None,None,None,None,None
3,Puzzle,Platformer,None,None,None,None,None,None
4,First Person Shooter,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
9681,Open World,Action-RPG,Role Playing,None,None,None,None,None
9682,Open World,Action-RPG,Role Playing,None,None,None,None,None
9683,Puzzle,Adventure,Platformer,None,None,None,None,None
9684,Puzzle,Adventure,Platformer,None,None,None,None,None


In [4]:
#Splitting the games with multiple genres
df = data_import.copy()
df = pd.concat([df, data_import["Genre"].str.split(',', expand=True )], axis=1)
df = df.melt(id_vars=["GameName", "Publisher"],value_vars=range(8))
df = df[df["value"].notna()]
df

,GameName,Publisher,variable,value
0,Q*Bert Rebooted,LOOT Interactive,0,Action
1,Q*Bert Rebooted (EU),LOOT Interactive,0,Action
2,Q.U.B.E. 2,Trapped Nerve Games,0,Puzzle
3,Q.U.B.E. Director's Cut,GRIP Digital,0,Puzzle
4,Quake (PS4),Bethesda Softworks,0,First Person Shooter
...,...,...,...,...
50351,Trailmakers,Flashbulb Games,5,Sandbox
50352,Trailmakers (JP),EXNOA,5,Sandbox
54204,Fallout 76,Bethesda Softworks,5,Third Person Shooter
63890,Fallout 76,Bethesda Softworks,6,Survival


In [5]:
px.histogram(df, x="value")

# EDA

In [6]:
data_import.dtypes


ID                                int64
GamePSID                          int64
GameName                         object
Publisher                        object
ReleaseYear                     float64
ReleaseDate              datetime64[ns]
Developer                        object
Genre                            object
Features                         object
Size                            float64
Medium                           object
Hardware                         object
CompletionTime(Hours)            object
is_Digital_game                   int64
is_Physical_game                  int64
OfficialWebsite                  object
dtype: object

## Averages

In [7]:

#games['Size'].mean()
#stats.trim_mean(games['Size'], 0.4)
data_import['Size'].median()

1392.64

## Numerical column describe

In [8]:
data_import.describe([.25])

,ID,GamePSID,ReleaseYear,Size,is_Digital_game,is_Physical_game
count,9686.000000,9686.000000,8952.000000,7.666000e+03,9686.0,9686.0
mean,4843.500000,7450.627813,2018.515304,5.207188e+03,0.0,0.0
std,2796.251688,4434.041993,2.014025,1.880542e+04,0.0,0.0
min,1.000000,-5590.000000,2009.000000,1.100000e+01,0.0,0.0
25%,2422.250000,5108.250000,2017.000000,3.922500e+02,0.0,0.0
50%,4843.500000,7972.500000,2019.000000,1.392640e+03,0.0,0.0
max,9686.000000,14229.000000,2022.000000,1.002496e+06,0.0,0.0


### Date analysis

### Size

In [9]:
data_import['Size'].max()
data_import['Size'].idxmax()

7435

### Box Plot

In [10]:
fig = px.box(data_import['Size']/10000, y = "Size", range_y= [0,3])
fig.show()


### Frequency table

In [11]:
state = data_import
binnedPopulation = pd.cut(state['Population'], 10)
binnedPopulation.value_counts()

KeyError: 'Population'